In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

def preprocess_image(image, label):
    image = image / 255.0
    return image, label

train_ds = image_dataset_from_directory(
    'dataset/train',
    image_size=(48 ,48),
    validation_split=0.2,
    seed=123,
    color_mode='grayscale',
    batch_size=32,
    subset='training'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset/train',
    image_size=(48 ,48),
    validation_split=0.2,
    seed=123,
    color_mode='grayscale',
    batch_size=32,
    subset='validation'
)

train_ds = train_ds.map(lambda x, y: preprocess_image(x, y))
val_ds = val_ds.map(lambda x, y: preprocess_image(x, y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(250).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.


In [15]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Dense, Input, Flatten 

model = Sequential([
    Input(shape=(48, 48, 1)),
    Conv2D(64, 5, activation='relu'),
    BatchNormalization(),
    Conv2D(64, 5, activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.3),

    Conv2D(128, 3, activation='relu'),
    BatchNormalization(),
    Conv2D(128, 3, activation='relu'), 
    BatchNormalization(), 
    MaxPooling2D(2),
    Dropout(0.4),

    Conv2D(256, 3, activation='relu'), 
    BatchNormalization(),
    Conv2D(256, 3, activation='relu'), 
    BatchNormalization(), 
    MaxPooling2D(2),
    Dropout(0.5),
    Flatten(),

    Dense(128, activation='relu'), 
    Dropout(0.5), 
    Dense(7, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [16]:
model.fit(
    train_ds,
    epochs=75, 
    batch_size=32, 
    validation_data=val_ds,
)

Epoch 1/75
718/718 [==============================] - 27s 36ms/step - loss: 1.9104 - accuracy: 0.2545 - val_loss: 1.6642 - val_accuracy: 0.3440
Epoch 2/75
718/718 [==============================] - 26s 36ms/step - loss: 1.6494 - accuracy: 0.3434 - val_loss: 1.5714 - val_accuracy: 0.3853
Epoch 3/75
718/718 [==============================] - 26s 36ms/step - loss: 1.5577 - accuracy: 0.3864 - val_loss: 1.5076 - val_accuracy: 0.4130
Epoch 4/75
718/718 [==============================] - 26s 36ms/step - loss: 1.4824 - accuracy: 0.4218 - val_loss: 1.4071 - val_accuracy: 0.4419
Epoch 5/75
718/718 [==============================] - 26s 36ms/step - loss: 1.4239 - accuracy: 0.4477 - val_loss: 1.4594 - val_accuracy: 0.4480
Epoch 6/75
718/718 [==============================] - 26s 36ms/step - loss: 1.3828 - accuracy: 0.4658 - val_loss: 1.3977 - val_accuracy: 0.4518
Epoch 7/75
718/718 [==============================] - 26s 36ms/step - loss: 1.3398 - accuracy: 0.4858 - val_loss: 1.2803 - val_accuracy:

In [13]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
     'dataset/test',
    image_size=(48 ,48),
    color_mode='grayscale',
    batch_size=32,
)

test_ds = test_ds.map(lambda x, y: preprocess_image(x, y))

AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().shuffle(250).prefetch(buffer_size=AUTOTUNE)

test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Found 7178 files belonging to 7 classes.
225/225 [==============================] - 10s 13ms/step - loss: 2.7288 - accuracy: 0.6057
Test Loss: 2.7287821769714355
Test Accuracy: 0.6057397723197937


In [14]:
model.save('fer_custom.h5')